# Aucnet Analysis

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
import seaborn as sns
import datetime

In [2]:
bid=pd.read_csv('all_bid.csv')
bid['bid_placed_on']=pd.to_datetime(bid['bid_placed_on'])

/Users/sydneybednar/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
auc_key=pd.read_excel('AuctionSellers.xlsx')
auc_key

,auction_id,username
0,1,aucnet
1,2,aucnet
2,3,aucnet
3,4,aucnet
4,5,aucnet
...,...,...
325,319,camille+ecoring
326,322,camille+ecoring
327,329,camille+ecoring
328,330,camille+ecoring


In [4]:
new=bid.merge(auc_key, how='left')
aucnet=new[new['username']=='aucnet']

In [5]:
grp=aucnet.groupby('auction_id')
day_df=pd.DataFrame(grp['day'].last())

dict={}
for i in grp.groups:
    end_day=day_df.loc[day_df.index==i]['day'][i]
    if end_day==6:
        dict[i]='aucnet low'
    elif end_day>=0:
        dict[i]='aucnet high'
        
new_auc_key=pd.DataFrame.from_dict(dict, orient='index')
aucnet=aucnet.merge(new_auc_key, left_on='auction_id', right_index=True, how='left')

In [6]:
aucnet['bid'].mean()

482.98263947933896

In [7]:
aucnet[0].value_counts()

aucnet high    39894
aucnet low     24793
Name: 0, dtype: int64

In [8]:
for i in range(2020,2022):
    for a in ['aucnet high', 'aucnet low']:
        mean_bid=aucnet[(aucnet[0]==a)&(aucnet['bid_placed_on'].dt.year==i)]['bid'].mean()
        median_bid=aucnet[(aucnet[0]==a)&(aucnet['bid_placed_on'].dt.year==i)]['bid'].median()
        mean_min=aucnet[(aucnet[0]==a)&(aucnet['bid_placed_on'].dt.year==i)]['min_bid'].mean()
        median_min=aucnet[(aucnet[0]==a)&(aucnet['bid_placed_on'].dt.year==i)]['min_bid'].median()
        print('the mean bid in '+str(i)+' for '+str(a)+': ${:.2f}'.format(mean_bid))
        print('the median bid in '+str(i)+' for '+str(a)+': ${:.2f}'.format(median_bid))
        print('the mean minimum bid in '+str(i)+' for '+str(a)+': ${:.2f}'.format(mean_min))
        print('the median minimum bid in '+str(i)+' for '+str(a)+': ${:.2f}'.format(median_min))
        print('\n')

the mean bid in 2020 for aucnet high: $529.35
the median bid in 2020 for aucnet high: $399.00
the mean minimum bid in 2020 for aucnet high: $301.49
the median minimum bid in 2020 for aucnet high: $164.79


the mean bid in 2020 for aucnet low: $319.46
the median bid in 2020 for aucnet low: $278.00
the mean minimum bid in 2020 for aucnet low: $173.33
the median minimum bid in 2020 for aucnet low: $150.93


the mean bid in 2021 for aucnet high: $818.11
the median bid in 2021 for aucnet high: $671.50
the mean minimum bid in 2021 for aucnet high: $512.42
the median minimum bid in 2021 for aucnet high: $397.50


the mean bid in 2021 for aucnet low: $302.56
the median bid in 2021 for aucnet low: $268.00
the mean minimum bid in 2021 for aucnet low: $119.26
the median minimum bid in 2021 for aucnet low: $93.64




In [9]:
for a in ['aucnet high', 'aucnet low']:
    mean_bid=aucnet[(aucnet[0]==a)&(aucnet['bid_placed_on']>datetime.datetime(2021,4,7))]['bid'].mean()
    median_bid=aucnet[(aucnet[0]==a)&(aucnet['bid_placed_on']>datetime.datetime(2021,4,7))]['bid'].median()
    mean_min=aucnet[(aucnet[0]==a)&(aucnet['bid_placed_on']>datetime.datetime(2021,4,7))]['min_bid'].mean()
    median_min=aucnet[(aucnet[0]==a)&(aucnet['bid_placed_on']>datetime.datetime(2021,4,7))]['min_bid'].median()
    print('the mean bid in the past two months for '+str(a)+': ${:.2f}'.format(mean_bid))
    print('the median bid in the past two months for '+str(a)+': ${:.2f}'.format(median_bid))
    print('the mean minimum bid in the past two months for '+str(a)+': ${:.2f}'.format(mean_min))
    print('the median minimum bid in the past two months for '+str(a)+': ${:.2f}'.format(median_min))
    print('\n')

the mean bid in the past two months for aucnet high: $699.74
the median bid in the past two months for aucnet high: $560.00
the mean minimum bid in the past two months for aucnet high: $411.19
the median minimum bid in the past two months for aucnet high: $309.01


the mean bid in the past two months for aucnet low: $321.79
the median bid in the past two months for aucnet low: $280.00
the mean minimum bid in the past two months for aucnet low: $131.61
the median minimum bid in the past two months for aucnet low: $101.92




In [10]:
top5=list(aucnet['brand_name'].value_counts().index[:5])

In [11]:
for a in ['aucnet high', 'aucnet low']:
    for b in top5: 
        mean_min=(aucnet[(aucnet[0]==a)&
                         (aucnet['bid_placed_on']>datetime.datetime(2021,4,7))&
                        (aucnet['brand_name']==b)]['min_bid'].mean())
        median_min=(aucnet[(aucnet[0]==a)&
                           (aucnet['bid_placed_on']>datetime.datetime(2021,4,7))&
                        (aucnet['brand_name']==b)]['min_bid'].median())
        count=len(aucnet[(aucnet[0]==a)&
                      (aucnet['bid_placed_on']>datetime.datetime(2021,4,7))&
                      (aucnet['brand_name']==b)].groupby(['brand_name', 'item_id']).count())
        print('the mean minimum bid in the past two months for '+str(b)+' in '+str(a)+': ${:.2f}'.format(mean_min))
        print('the median minimum bid in the past two months for '+str(b)+' in '+str(a)+': ${:.2f}'.format(median_min))
        print(count)
        print('\n')

the mean minimum bid in the past two months for Louis Vuitton in aucnet high: $394.90
the median minimum bid in the past two months for Louis Vuitton in aucnet high: $346.70
2234


the mean minimum bid in the past two months for Chanel in aucnet high: $502.49
the median minimum bid in the past two months for Chanel in aucnet high: $251.11
1165


the mean minimum bid in the past two months for Gucci in aucnet high: $245.68
the median minimum bid in the past two months for Gucci in aucnet high: $203.23
209


the mean minimum bid in the past two months for Hermès in aucnet high: $268.17
the median minimum bid in the past two months for Hermès in aucnet high: $92.38
284


the mean minimum bid in the past two months for Christian Dior in aucnet high: $599.88
the median minimum bid in the past two months for Christian Dior in aucnet high: $374.53
79


the mean minimum bid in the past two months for Louis Vuitton in aucnet low: $154.72
the median minimum bid in the past two months for Louis V

In [12]:
for i in range(2020,2022):
    for a in ['aucnet high', 'aucnet low']:
        for b in top5:
            mean_min=(aucnet[(aucnet[0]==a)&
                             (aucnet['bid_placed_on'].dt.year==i)&
                            (aucnet['brand_name']==b)]['min_bid'].mean())
            median_min=(aucnet[(aucnet[0]==a)&
                               (aucnet['bid_placed_on'].dt.year==i)&
                            (aucnet['brand_name']==b)]['min_bid'].median())
            count=len(aucnet[(aucnet[0]==a)&
                      (aucnet['bid_placed_on'].dt.year==i)&
                      (aucnet['brand_name']==b)].groupby(['brand_name', 'item_id']).count())
            print('the mean minimum bid in '+str(i)+' for '+str(b)+' in '+str(a)+': ${:.2f}'.format(mean_min))
            print('the median minimum bid in '+str(i)+' for '+str(b)+' in '+str(a)+': ${:.2f}'.format(median_min))
            print(count)
            print('\n')

the mean minimum bid in 2020 for Louis Vuitton in aucnet high: $282.88
the median minimum bid in 2020 for Louis Vuitton in aucnet high: $193.87
13760


the mean minimum bid in 2020 for Chanel in aucnet high: $368.33
the median minimum bid in 2020 for Chanel in aucnet high: $137.42
5650


the mean minimum bid in 2020 for Gucci in aucnet high: $223.89
the median minimum bid in 2020 for Gucci in aucnet high: $117.30
1510


the mean minimum bid in 2020 for Hermès in aucnet high: $411.99
the median minimum bid in 2020 for Hermès in aucnet high: $114.89
868


the mean minimum bid in 2020 for Christian Dior in aucnet high: $236.21
the median minimum bid in 2020 for Christian Dior in aucnet high: $116.10
496


the mean minimum bid in 2020 for Louis Vuitton in aucnet low: $181.74
the median minimum bid in 2020 for Louis Vuitton in aucnet low: $166.32
3624


the mean minimum bid in 2020 for Chanel in aucnet low: $162.72
the median minimum bid in 2020 for Chanel in aucnet low: $141.93
957


the m

In [13]:
aucnet=aucnet[aucnet['bid_placed_on'].dt.year==2021]

In [30]:
aucnet['hour_date']=aucnet['bid_placed_on'].dt.strftime('[%m/%d, %H]')
aucnet['date']=aucnet['bid_placed_on'].dt.date
key=aucnet.groupby('auction_id')['hour_date'].last().reset_index()

In [15]:
new=key.merge(aucnet, how='left')
new_2=new.groupby(['auction_id','buyer'])['bid'].count().reset_index()
new_2=new_2.rename(columns={'bid':'bids_in_last_hour'})

In [16]:
new_2

,auction_id,buyer,bids_in_last_hour
0,250,smitten luxury,1
1,250,style encore maple grove,7
2,251,paris diva,7
3,251,smitten luxury,1
4,252,bagriculture,61
...,...,...,...
99,325,smitten luxury,3
100,326,ds darling deals,1
101,331,paris diva,14
102,332,curateur,4


In [17]:
total=aucnet.groupby(['auction_id','buyer'])['bid'].count().reset_index()
total.groupby('auction_id')['buyer'].count()

auction_id
250     9
251    20
252    26
253    31
256    34
257    20
259    29
260    26
264    30
265    23
267    33
268    31
272    28
273    19
275    28
276    28
278    25
279    20
281    20
282    16
286    30
287    20
289    28
290    24
293    23
294    22
298    21
299    22
300    35
301    26
304    36
305    18
309    26
310    17
314    42
315    23
320    30
321    15
323    22
324    14
325    24
326    16
331    27
332    17
Name: buyer, dtype: int64

In [18]:
new_2.groupby('auction_id')['buyer'].count()/total.groupby('auction_id')['buyer'].count()

auction_id
250    0.222222
251    0.100000
252    0.192308
253    0.193548
256    0.088235
257    0.050000
259    0.103448
260    0.076923
264    0.100000
265    0.086957
267    0.060606
268    0.032258
272    0.035714
273    0.263158
275    0.071429
276    0.107143
278    0.120000
279    0.100000
281    0.150000
282    0.062500
286    0.066667
287    0.100000
289    0.107143
290    0.041667
293    0.086957
294    0.090909
298    0.142857
299    0.227273
300    0.171429
301    0.038462
304    0.083333
305    0.055556
309    0.192308
310    0.117647
314    0.023810
315    0.043478
320    0.033333
321    0.066667
323    0.136364
324    0.071429
325    0.083333
326    0.062500
331    0.037037
332    0.117647
Name: buyer, dtype: float64

In [19]:
perc=(new_2.groupby('auction_id')['buyer'].count()/total.groupby('auction_id')['buyer'].count()*100).mean()

In [28]:
print('the avg percent of bidders who bid within the last hour of aucnet auction 2021: {:.2f}%'.format(perc))

the avg percent of bidders who bid within the last hour of aucnet auction 2021: 10.03%


In [21]:
key_date=aucnet.groupby('auction_id')['date'].last().reset_index()
key_date

,auction_id,date
0,250,2021-01-03
1,251,2021-01-04
2,252,2021-01-10
3,253,2021-01-11
4,256,2021-01-17
5,257,2021-01-18
6,259,2021-01-24
7,260,2021-01-25
8,264,2021-01-31
9,265,2021-02-01


In [22]:
new_date=key_date.merge(aucnet, how='left')
new_date2=new_date.groupby(['auction_id','buyer'])['bid'].count().reset_index()
new_date2=new_date2.rename(columns={'bid':'bids_on_last_day'})

In [23]:
new_date2.groupby('auction_id')['buyer'].count()/total.groupby('auction_id')['buyer'].count()*100

auction_id
250    55.555556
251    45.000000
252    38.461538
253    45.161290
256    47.058824
257    30.000000
259    44.827586
260    42.307692
264    30.000000
265    30.434783
267    39.393939
268    22.580645
272    21.428571
273    42.105263
275    39.285714
276    35.714286
278    44.000000
279    50.000000
281    40.000000
282    50.000000
286    43.333333
287    15.000000
289    35.714286
290     4.166667
293    47.826087
294    45.454545
298    38.095238
299    54.545455
300    37.142857
301    38.461538
304    41.666667
305    44.444444
309    50.000000
310    29.411765
314    14.285714
315    21.739130
320     3.333333
321    26.666667
323    63.636364
324    35.714286
325    45.833333
326    18.750000
331     7.407407
332    52.941176
Name: buyer, dtype: float64

In [27]:
perc_date=(new_date2.groupby('auction_id')['buyer'].count()/total.groupby('auction_id')['buyer'].count()*100).mean()
print('the avg percent of bidders who bid within the last day of aucnet auction 2021: {:.2f}%'.format(perc_date))

the avg percent of bidders who bid within the last day of aucnet auction 2021: 36.57%


In [35]:
# new_2.to_excel('aucnet_last_hour.xlsx')
# new_date2.to_excel('aucnet_last_day.xlsx')